In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
import nltk

import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from tensorflow.python.client import device_lib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional

nltk.download("stopwords")
nltk.download('wordnet')
stop_words = set(stopwords.words("english"))
lemmatizer= WordNetLemmatizer()



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6257383121244799639
xla_global_id: -1
]


In [3]:
train =  pd.read_csv('/content/drive/MyDrive/ML/train.txt',sep = ';',names=['text','label'])
test = pd.read_csv('/content/drive/MyDrive/ML/test.txt',sep = ';',names=['text','label'])
val = pd.read_csv('/content/drive/MyDrive/ML/val.txt',sep = ';',names=['text','label'])


In [4]:
train.shape

(16000, 2)

In [5]:
train.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [6]:
val.head()

,text,label
0,im feeling quite sad and sorry for myself but ...,sadness
1,i feel like i am still looking at a blank canv...,sadness
2,i feel like a faithful servant,love
3,i am just feeling cranky and blue,anger
4,i can have for a treat or if i am feeling festive,joy


In [7]:
train.label.value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: label, dtype: int64

In [8]:
test.label.value_counts()

joy         695
sadness     581
anger       275
fear        224
love        159
surprise     66
Name: label, dtype: int64

In [9]:
val.label.value_counts()

joy         704
sadness     550
anger       275
fear        212
love        178
surprise     81
Name: label, dtype: int64

In [10]:
train[train['text'].duplicated()==True]

,text,label
5067,i feel on the verge of tears from weariness i ...,joy
6133,i still feel a craving for sweet food,love
6563,i tend to stop breathing when i m feeling stre...,anger
7623,i was intensely conscious of how much cash i h...,sadness
7685,im still not sure why reilly feels the need to...,surprise
8246,i am not amazing or great at photography but i...,love
9596,ive also made it with both sugar measurements ...,joy
9687,i had to choose the sleek and smoother feel of...,joy
9769,i often find myself feeling assaulted by a mul...,sadness
9786,i feel im being generous with that statement,joy


In [11]:
train[train['text'] == train.iloc[6133]['text']]

,text,label
4476,i still feel a craving for sweet food,joy
6133,i still feel a craving for sweet food,love


In [12]:
index = train[train['text'].duplicated() == True].index
train.drop(index, axis = 0, inplace = True)
train.reset_index(inplace=True, drop = True)

In [13]:
val[val['text'].duplicated()==True]

,text,label
603,i have had several new members tell me how com...,joy
1993,i feel so tortured by it,anger


In [14]:
val[val['text'] == val.iloc[603]['text']]

,text,label
300,i have had several new members tell me how com...,love
603,i have had several new members tell me how com...,joy


In [15]:
index = val[val['text'].duplicated() == True].index
val.drop(index, axis = 0, inplace = True)
val.reset_index(inplace=True, drop = True)

In [16]:
test[test['text'].duplicated()==True]

,text,label


In [17]:
index = test[test['text'].duplicated() == True].index
test.drop(index, axis = 0, inplace = True)
test.reset_index(inplace=True, drop = True)

In [18]:
def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]

    return " " .join(text)

def remove_stop_words(text):

    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):

    text = text.split()

    text=[y.lower() for y in text]

    return " " .join(text)

def Removing_punctuations(text):
    # Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )

    # remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def Removing_urls(text):
    url_pattern = re.compile(r'http?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F700-\U0001F77F"  # alchemical symbols
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= Removing_numbers(sentence)
    sentence= Removing_punctuations(sentence)
    sentence= Removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence
def normalize_text(df):
    df.text=df.text.apply(lambda text : lower_case(text))
    df.text=df.text.apply(lambda text : remove_stop_words(text))
    df.text=df.text.apply(lambda text : Removing_numbers(text))
    df.text=df.text.apply(lambda text : Removing_punctuations(text))
    df.text=df.text.apply(lambda text : Removing_urls(text))
    df.text=df.text.apply(lambda text : lemmatization(text))
    df.text=df.text.apply(lambda text : remove_emojis(text))
    return df

In [19]:
train= normalize_text(train)
test= normalize_text(test)
val= normalize_text(val)

In [20]:
x_train = train['text']
y_train = train['label']

x_val = val['text']
y_val = val['label']

x_test = test['text']
y_test = test['label']

In [21]:
print(set(y_train))

{'surprise', 'love', 'sadness', 'fear', 'anger', 'joy'}


In [22]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
y_val = le.transform(y_val)

In [23]:
print(set(y_train))

{0, 1, 2, 3, 4, 5}


In [24]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

In [25]:
print(y_train)

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [26]:
tokenizer = Tokenizer(oov_token='UNK')
tokenizer.fit_on_texts(pd.concat([x_train, x_test], axis=0))

In [27]:
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_test = tokenizer.texts_to_sequences(x_test)
sequences_val = tokenizer.texts_to_sequences(x_val)

In [28]:
maxlen = max([len(t) for t in train['text']])
maxlen

229

In [29]:
x_train = pad_sequences(sequences_train, maxlen=maxlen, truncating='pre')
x_test = pad_sequences(sequences_test, maxlen=maxlen, truncating='pre')
x_val = pad_sequences(sequences_val, maxlen=maxlen, truncating='pre')

vocabSize = len(tokenizer.index_word) + 1
print(f"Vocabulary size = {vocabSize}")

Vocabulary size = 14325


In [30]:
sequences_train[0]

[53, 2, 531]

In [31]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [ ]:
path_to_glove_file = '/content/drive/MyDrive/ML/glove.6B.200d.txt'
num_tokens = vocabSize
embedding_dim = 200 #features
hits = 0
misses = 0
embeddings_index = {}

In [ ]:
with open(path_to_glove_file,encoding="utf8") as f: # f = open('salkdsandkj.pdf')
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
print("Found %s word vectors." % len(embeddings_index))

# Assign word vectors to our dictionary/vocabulary
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
print(hits/vocabSize * 100)

In [ ]:
adam = Adam(learning_rate=0.005)

model = Sequential()
model.add(Embedding(vocabSize,embedding_dim, input_length=x_train.shape[1], weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 229, 200)          2865000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 229, 512)          935936    
 onal)                                                           
                                                                 
 bidirectional_4 (Bidirecti  (None, 229, 256)          656384    
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                      

In [ ]:
callback = EarlyStopping(
    monitor="val_loss",
    patience=4,
    restore_best_weights=True,
)

In [ ]:
model.fit(x_train,
                    y_train,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    batch_size=255,
                    epochs=12,
                    callbacks=[callback]
                   )

Epoch 1/12
63/63 [==============================] - 293s 5s/step - loss: 0.1038 - accuracy: 0.9482 - val_loss: 0.1292 - val_accuracy: 0.9379
Epoch 2/12
63/63 [==============================] - 282s 4s/step - loss: 0.0921 - accuracy: 0.9549 - val_loss: 0.1349 - val_accuracy: 0.9354
Epoch 3/12
63/63 [==============================] - 280s 4s/step - loss: 0.0875 - accuracy: 0.9582 - val_loss: 0.1301 - val_accuracy: 0.9349
Epoch 4/12
63/63 [==============================] - 285s 5s/step - loss: 0.0977 - accuracy: 0.9540 - val_loss: 0.1339 - val_accuracy: 0.9379
Epoch 5/12
63/63 [==============================] - 279s 4s/step - loss: 0.0881 - accuracy: 0.9565 - val_loss: 0.1384 - val_accuracy: 0.9364


In [ ]:
model.save('/content/drive/MyDrive/ML/ER_2.keras')

In [ ]:
model.evaluate(x_val, y_val)

63/63 [==============================] - 25s 403ms/step - loss: 0.1292 - accuracy: 0.9379


[0.12924666702747345, 0.9379379153251648]

In [ ]:
sentence= 'GET LOST !'
print(sentence)
sentence = normalized_sentence(sentence)
sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))[0]
proba =  np.max(model.predict(sentence))
print(f"{result} : {proba}\n\n")

GET LOST !
1/1 [==============================] - 0s 363ms/step
sadness : 0.939849317073822




In [32]:
loaded_model = load_model('/content/drive/MyDrive/ML/ER.keras')

In [35]:
sentence = 'I am so happy for you'
print(sentence)
sentence = normalized_sentence(sentence)
sentence = tokenizer.texts_to_sequences([sentence])
sentence = pad_sequences(sentence, maxlen=229, truncating='pre')
result = le.inverse_transform(np.argmax(loaded_model.predict(sentence), axis=-1))[0]
proba =  np.max(loaded_model.predict(sentence))
print(f"{result} : {proba}\n\n")

I am so happy for you
1/1 [==============================] - 0s 478ms/step
joy : 0.826305091381073


